# YOLO Object Detection for Image Analysis

This notebook demonstrates the use of YOLO (You Only Look Once) for object detection in images. We'll use the YOLOv8 model to detect objects in various images and analyze the results.

## Overview
1. Setup and Dependencies
2. Loading YOLOv8 Model
3. Object Detection on Sample Images
4. Analyzing and Visualizing Results
5. Advanced Applications

## 1. Setup and Dependencies

First, we'll install the necessary libraries and import them.

In [ ]:
# Install required packages
!pip install ultralytics
!pip install supervision

In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
from ultralytics import YOLO
import supervision as sv

# Set display parameters for better visualization
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['axes.grid'] = False

## 2. Loading YOLOv8 Model

Here we load the pre-trained YOLOv8 model. YOLOv8 is a state-of-the-art object detection model that offers an excellent balance between speed and accuracy.

In [ ]:
# Load a pre-trained YOLO model (YOLOv8n)
model = YOLO("yolov8n.pt")

# Display model information
print(f"Model loaded: {model.info()}")
print(f"Model task: {model.task}")

## 3. Object Detection on Sample Images

Now we'll perform object detection on sample images from different sources. We'll start with a basic example and then analyze more complex images.

### 3.1 Basic Object Detection

In [ ]:
# Sample image URL
IMAGE_URL = "https://ultralytics.com/images/bus.jpg"

# Download and display the original image
!wget -q -O bus.jpg {IMAGE_URL}

# Display the original image
plt.figure(figsize=(12, 8))
img = plt.imread("bus.jpg")
plt.imshow(img)
plt.title("Original Image", fontsize=16)
plt.axis('off')
plt.show()

In [ ]:
# Perform object detection on the image
results = model("bus.jpg")[0]

# Display the image with detections
detections = sv.Detections.from_yolov8(results)
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Read the image with OpenCV for annotation
img = cv2.imread("bus.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

# Annotate the image
annotated_image = bounding_box_annotator.annotate(img, detections)
annotated_image = label_annotator.annotate(annotated_image, detections)

# Display the annotated image
plt.figure(figsize=(14, 10))
plt.imshow(annotated_image)
plt.title("Object Detection Results", fontsize=16)
plt.axis('off')
plt.show()

### 3.2 Analysis of Detection Results

Let's analyze the results of our object detection to understand what objects were detected, along with their confidence scores and bounding box coordinates.

In [ ]:
# Analyze the detection results
print(f"Number of objects detected: {len(results.boxes)}")

# Extract class names from the model's names dictionary
class_names = model.names

# Create a summary of detected objects
detected_classes = {}
for box in results.boxes:
    class_id = int(box.cls[0].item())
    class_name = class_names[class_id]
    confidence = box.conf[0].item()
    
    if class_name in detected_classes:
        detected_classes[class_name].append(round(confidence, 3))
    else:
        detected_classes[class_name] = [round(confidence, 3)]

# Display summary
print("\nDetected Objects:")
for class_name, confidences in detected_classes.items():
    print(f"- {class_name}: {len(confidences)} instances, Confidence: {confidences}")

### 3.3 Object Detection on Additional Images

Let's test our model on more images to verify its performance across different scenarios.

In [ ]:
# List of image URLs to analyze
image_urls = [
    "https://ultralytics.com/images/zidane.jpg",
    "https://ultralytics.com/images/person.jpg"
]

# Function to process and visualize detections
def process_image(url, index):
    # Download image
    image_name = f"image_{index}.jpg"
    !wget -q -O {image_name} {url}
    
    # Run detection
    results = model(image_name)[0]
    
    # Prepare visualization
    img = cv2.imread(image_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    detections = sv.Detections.from_yolov8(results)
    annotated_image = bounding_box_annotator.annotate(img, detections)
    annotated_image = label_annotator.annotate(annotated_image, detections)
    
    # Count detections by class
    classes = {}
    for box in results.boxes:
        class_id = int(box.cls[0].item())
        class_name = model.names[class_id]
        if class_name in classes:
            classes[class_name] += 1
        else:
            classes[class_name] = 1
    
    # Display results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 9))
    
    # Original image
    ax1.imshow(img)
    ax1.set_title("Original Image", fontsize=14)
    ax1.axis('off')
    
    # Annotated image
    ax2.imshow(annotated_image)
    ax2.set_title(f"Objects Detected: {', '.join([f'{c} ({n})' for c, n in classes.items()])}", fontsize=14)
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return results, classes

# Process each image
for i, url in enumerate(image_urls):
    print(f"\nImage {i+1}:")
    results, classes = process_image(url, i+1)
    print(f"Detected: {classes}")

## 4. Advanced Analysis: Custom Images

Now let's analyze any custom images. You can upload your images to the notebook environment and run object detection on them.

In [ ]:
# Upload your custom image
from google.colab import files
uploaded = files.upload()

# Process uploaded images
for filename in uploaded.keys():
    print(f"Processing {filename}...")
    
    # Run object detection
    results = model(filename)[0]
    
    # Prepare visualization
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    detections = sv.Detections.from_yolov8(results)
    annotated_image = bounding_box_annotator.annotate(img, detections)
    annotated_image = label_annotator.annotate(annotated_image, detections)
    
    # Display images
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 9))
    
    # Original image
    ax1.imshow(img)
    ax1.set_title("Original Image", fontsize=14)
    ax1.axis('off')
    
    # Annotated image
    ax2.imshow(annotated_image)
    ax2.set_title("Detected Objects", fontsize=14)
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detection statistics
    print(f"\nDetection Results for {filename}:")
    detected_objects = {}
    for box in results.boxes:
        class_id = int(box.cls[0].item())
        class_name = model.names[class_id]
        confidence = round(box.conf[0].item(), 2)
        
        if class_name in detected_objects:
            detected_objects[class_name].append(confidence)
        else:
            detected_objects[class_name] = [confidence]
    
    # Print summary
    for obj_class, confidences in detected_objects.items():
        avg_conf = sum(confidences) / len(confidences)
        print(f"- {obj_class}: {len(confidences)} instances, Avg. Confidence: {avg_conf:.2f}")

## 5. Advanced Applications

### 5.1 Video Object Detection

YOLO can also perform object detection on videos. Let's implement a function to process videos frame by frame.

In [ ]:
# Function to process a video file
def process_video(video_path, output_path, save_output=True, display_frames=False):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video properties: {width}x{height} @ {fps} fps, {total_frames} frames total")
    
    # Create video writer if saving output
    if save_output:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Setup annotators
    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()
    
    # Process frames
    frame_count = 0
    
    while True:
        # Read frame
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_count += 1
        if frame_count % 10 == 0:  # Print status every 10 frames
            print(f"Processing frame {frame_count}/{total_frames}")
        
        # Run detection on frame
        results = model(frame)[0]
        
        # Convert detections to supervision format
        detections = sv.Detections.from_yolov8(results)
        
        # Annotate frame
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # For display
        annotated_frame = bounding_box_annotator.annotate(frame_rgb, detections)
        annotated_frame = label_annotator.annotate(annotated_frame, detections)
        
        # Convert back to BGR for video saving
        annotated_frame_bgr = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)
        
        # Save frame to output video
        if save_output:
            out.write(annotated_frame_bgr)
        
        # Display sample frames if requested
        if display_frames and frame_count % 30 == 0:  # Display every 30th frame
            plt.figure(figsize=(10, 6))
            plt.imshow(annotated_frame)
            plt.title(f"Frame {frame_count}")
            plt.axis('off')
            plt.show()
    
    # Release resources
    cap.release()
    if save_output:
        out.release()
        
    print(f"Video processing completed. Output saved to: {output_path}")

# Example usage (commented out as we don't have video files in this session)
# You can upload a video using files.upload() and then process it

# process_video('input_video.mp4', 'output_video.mp4', save_output=True, display_frames=True)

### 5.2 Statistics and Analysis

Let's create a function to analyze detection statistics across multiple images.

In [ ]:
# Function to analyze detection statistics
def analyze_detections(results_list, image_names=None):
    if image_names is None:
        image_names = [f"Image {i+1}" for i in range(len(results_list))]
    
    # Collect statistics
    all_stats = []
    
    for i, results in enumerate(results_list):
        image_stats = {"name": image_names[i], "objects": {}}
        
        # Count objects by class
        for box in results.boxes:
            class_id = int(box.cls[0].item())
            class_name = model.names[class_id]
            confidence = float(box.conf[0].item())
            
            if class_name not in image_stats["objects"]:
                image_stats["objects"][class_name] = {
                    "count": 0,
                    "confidences": []
                }
                
            image_stats["objects"][class_name]["count"] += 1
            image_stats["objects"][class_name]["confidences"].append(confidence)
        
        all_stats.append(image_stats)
    
    # Calculate overall statistics
    class_counts = {}
    
    for stats in all_stats:
        for class_name, data in stats["objects"].items():
            if class_name not in class_counts:
                class_counts[class_name] = 0
            class_counts[class_name] += data["count"]
    
    # Display statistics
    print("Detection Statistics:")
    print("-" * 40)
    
    for i, stats in enumerate(all_stats):
        print(f"\n{stats['name']}:")
        for class_name, data in stats["objects"].items():
            avg_conf = sum(data["confidences"]) / len(data["confidences"])
            print(f"  - {class_name}: {data['count']} instances, Avg. Confidence: {avg_conf:.2f}")
    
    print("\nOverall Statistics:")
    print("-" * 40)
    total_objects = sum(class_counts.values())
    print(f"Total objects detected: {total_objects}")
    
    # Sort classes by occurrence count
    sorted_classes = sorted(class_counts.items(), key=lambda x: x[1], reverse=True)
    for class_name, count in sorted_classes:
        print(f"  - {class_name}: {count} instances ({count/total_objects*100:.1f}%)")
    
    # Create visualization
    plt.figure(figsize=(12, 6))
    
    # Plot top 10 classes
    top_classes = sorted_classes[:10] if len(sorted_classes) > 10 else sorted_classes
    
    classes = [c[0] for c in top_classes]
    counts = [c[1] for c in top_classes]
    
    plt.bar(classes, counts, color='skyblue')
    plt.title("Top Detected Object Classes", fontsize=15)
    plt.xlabel("Object Class")
    plt.ylabel("Count")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return all_stats, class_counts

## 6. Conclusion and Next Steps

In this notebook, we've demonstrated how to use YOLOv8 for object detection in images. We've covered:

1. Setting up and loading the YOLO model
2. Performing object detection on images
3. Visualizing and analyzing detection results
4. Advanced applications like video processing
5. Statistical analysis of detections

### Next Steps

To extend this project, you could:
- Fine-tune the YOLO model on your own dataset
- Implement real-time object detection using a webcam
- Develop specific applications based on detection results (e.g., counting people, traffic analysis)
- Combine with tracking algorithms for object tracking in videos
- Export the model for deployment in production environments